In [ ]:
import numpy as np
import os
from utils import *

In [ ]:
%matplotlib notebook

## Objectives:

1. **Object detection with YOLOv2**
2. **Object tracking with SORT** 
3. **Object classification**
4. **Camera control and planning**

In [ ]:
#############################
### Simulation Parameters ###
#############################
x_c = np.array([0, 20, 20])
f = 10
sensor_w = 6
sensor_h = 4

u_scale = 100/180*np.pi
t_max = 20
fps = 10
x_max = 40

In [ ]:
def update(f, suffix=""):
    if f == 0:
        phi_t = phi_0
    t = f/fps
    if f >= t_max*fps:
        return
    psi_t = psi_0 + np.sum(phi_dots[:f, 0])/fps
    phi_t = phi_0 + np.sum(phi_dots[:f, 1])/fps
    # Update objects
    camera_fov.update_FOV(psi=psi_t, phi=phi_t)
    camera_fov.psi_dot = phi_dots[f, 0]
    camera_fov.phi_dot = phi_dots[f, 1]
    for v in vehicles:
        if v.is_visible:
            v.update_vehicle(t)
            v.update_vehicle_virtual(camera_fov, t)
            v_data = v.get_vehicle_data(t)
            if np.max(v_data[:, 0])<=0 or np.min(v_data[:, 0])>=x_max or np.max(v_data[:, 1])<=0 or np.min(v_data[:, 1])>=x_max:
                v.set_visible(False)
    if t >= t_max/2:
        for v in vehicles2:
            if t==t_max/2:
                v.set_visible(True)
            if v.is_visible:
                v.update_vehicle(t-t_max/2)
                v.update_vehicle_virtual(camera_fov, t-t_max/2)
                v_data = v.get_vehicle_data(t-t_max/2)
                if np.max(v_data[:, 0])<=0 or np.min(v_data[:, 0])>=x_max or np.max(v_data[:, 1])<=0 or np.min(v_data[:, 1])>=x_max:
                    v.set_visible(False)
    for b in background:
        b.update_vehicle_virtual(camera_fov)
    # Detect objects
    obj_detector.detect_objects()
    # Update angle plot
    psi_data[1, f] = psi_t
    psi_line.set_data(psi_data[:, :(f+1)])
    phi_data[1, f] = phi_t
    phi_line.set_data(phi_data[:, :(f+1)])
    # Update u_plot
    psi_dot, phi_dot = psi_phi_dot(t, t_max)
    u1_data[1, f] = psi_dot/u_scale
    u1_line.set_data(u1_data[:, :(f+1)])
    u2_data[1, f] = phi_dot/u_scale
    u2_line.set_data(u2_data[:, :(f+1)])
    if (f+1)%fps == 0:
        title.set_text(
            'PT Camera Simulation, time={t:0.0f}s\nApplication: {app_name}'.format(t=t, app_name=app_name))

In [ ]:
phi_dots_dynamic = np.zeros((fps*t_max, 2))
phi_min = np.pi - np.pi/3.4
def update_dynamic(f, suffix=""):
    if f == 0:
        phi_t = phi_0
    t = f/fps
    if f >= t_max*fps:
        return
    # Get dynamic phi_dots
    print(f"Tracking id {obj_detector.tracking_id}")
    if obj_detector.tracking_id is not None:
        # Get inertial velocity
        v_x = obj_detector.objects[obj_detector.tracking_id][2]
        v_y = obj_detector.objects[obj_detector.tracking_id][3]
        # Convert to psi_dot, phi_dot
        i_vel_to_psi_phi_dot_xf = inertial_velocity_to_psi_phi_dot_gen(
            camera_fov.psi, camera_fov.phi, f, x_c)
        psi_dot, phi_dot = i_vel_to_psi_phi_dot_xf([v_x, v_y])
        psi_dot *= fps
        phi_dot *= fps
        if camera_fov.phi <= phi_min and phi_dot < 0:
            # Check constraints
            phi_dot = phi_dots[f, :][1]
        # Change virtual FOV text
        ax2.set_title(f"Virtual FOV: Tracking ID {obj_detector.tracking_id}")
    else:
        psi_dot, phi_dot = phi_dots[f, :]
        ax2.set_title("Virtual FOV: Surveillance")
    phi_dots_dynamic[f] = psi_dot, phi_dot
    psi_t = psi_0 + np.sum(phi_dots_dynamic[:f+1, 0])/fps
    phi_t = phi_0 + np.sum(phi_dots_dynamic[:f+1, 1])/fps
    # Update angles
    camera_fov.update_FOV(psi=psi_t, phi=phi_t)
    camera_fov.psi_dot = phi_dots_dynamic[f, 0]
    camera_fov.phi_dot = phi_dots_dynamic[f, 1]
    # Update objects
    for v in vehicles:
        if v.is_visible:
            v.update_vehicle(t)
            v.update_vehicle_virtual(camera_fov, t)
            v_data = v.get_vehicle_data(t)
            if np.max(v_data[:, 0])<=0 or np.min(v_data[:, 0])>=x_max or np.max(v_data[:, 1])<=0 or np.min(v_data[:, 1])>=x_max:
                v.set_visible(False)
    if t >= t_max/2:
        for v in vehicles2:
            if t==t_max/2:
                v.set_visible(True)
            if v.is_visible:
                v.update_vehicle(t-t_max/2)
                v.update_vehicle_virtual(camera_fov, t-t_max/2)
                v_data = v.get_vehicle_data(t-t_max/2)
                if np.max(v_data[:, 0])<=0 or np.min(v_data[:, 0])>=x_max or np.max(v_data[:, 1])<=0 or np.min(v_data[:, 1])>=x_max:
                    v.set_visible(False)
    for b in background:
        b.update_vehicle_virtual(camera_fov)
    # Detect objects
    obj_detector.detect_objects()
    # Update angle plot
    psi_data[1, f] = psi_t
    psi_line.set_data(psi_data[:, :(f+1)])
    phi_data[1, f] = phi_t
    phi_line.set_data(phi_data[:, :(f+1)])
    # Update u_plot
    #psi_dot, phi_dot = psi_phi_dot(t, t_max)
    u1_data[1, f] = psi_dot/u_scale
    u1_line.set_data(u1_data[:, :(f+1)])
    u2_data[1, f] = phi_dot/u_scale
    u2_line.set_data(u2_data[:, :(f+1)])
    if (f+1)%fps == 0:
        title.set_text(
            'PT Camera Simulation, time={t:0.0f}s\nApplication: {app_name}'.format(t=t, app_name=app_name))
    if f == 140:
        plt.savefig("simulations/PT_camera_simulation_wildlife_16_9_yolo_sort_classify_track_snapshot5.png", dpi=300, bbox_inches="tight")

In [ ]:
def psi_phi_dots_simulation(t, t_max):
    if t <= t_max/2 - 1:
        return np.pi/(t_max/2-1), 0
    elif t > t_max/2 - 1 and t <= t_max/2 + 1:
        return - np.pi/3/2, (np.pi/5 - np.pi/3.6)/2
    else:
        return -np.pi/3/(t_max/2-1), 0

In [ ]:
sim_configs = [
    [
        [
            [
                lambda ax, ax2: LinearVehicle(ax, ax2, x_offset=1, y_offset=1, v_l=2, v_w=4, vel_dir=np.array([2, 1.4])*3, color='gray'),
                lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=1, y_offset=39, vel_dir=np.array([1, -1])*2, color='burlywood'),
                lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=25, y_offset=1,  vel_dir=np.array([-1, 4]), color='saddlebrown')
            ],
            [
                lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=1, y_offset=20, vel_dir=np.array([1, 0.1])*5, color='gray', is_visible=False),
                lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=20, y_offset=1, vel_dir=np.array([1, 4])*1.5, color='burlywood', is_visible=False),
                lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=25, y_offset=39,  vel_dir=np.array([-1, -4])*2, color='black', is_visible=False)
            ]
        ],
        [
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=3, v_w=4, x_offset=30, y_offset=25,  vel_dir=np.array([0.5, 1]), color="sienna"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=5, v_w=2, x_offset=30, y_offset=10,  vel_dir=np.array([1, 1]), color="steelblue"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=4, v_w=4, x_offset=10, y_offset=10, vel_dir=np.array([0, 0]), color="darkgreen"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=3, v_w=4, x_offset=10, y_offset=30, vel_dir=np.array([0, 0]), color="slategray"),
        ],
        [
            {"suffix":"conf_0_it_0", "psi_0": 0, "phi_0": np.pi-np.pi/5, 
             "psi_phi_dot": psi_phi_dots_simulation},
        ]
    ],
    [
        [
            [
                lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=1, y_offset=20, vel_dir=np.array([1, 0.1])*7, color='gray'),
                lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=1, y_offset=39, vel_dir=np.array([0.85, -1])*5, color='burlywood'),
                lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=39, y_offset=1,  vel_dir=np.array([-1, 2])*2, color='saddlebrown'),
            ],
            [
                lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=1, y_offset=20, vel_dir=np.array([1, 0.1])*5, color='gray', is_visible=False),
                lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=20, y_offset=1, vel_dir=np.array([1, 4])*1.5, color='burlywood', is_visible=False),
                lambda ax, ax2: LinearVehicle(ax, ax2, v_l=2, v_w=4, x_offset=15, y_offset=39,  vel_dir=np.array([-0.1, -4])*1.5, color='black', is_visible=False),
            ]
    
        ],
        [
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=3, v_w=4, x_offset=30, y_offset=25,  vel_dir=np.array([0.5, 1]), color="sienna"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=5, v_w=2, x_offset=30, y_offset=10,  vel_dir=np.array([1, 1]), color="steelblue"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=4, v_w=4, x_offset=10, y_offset=10, vel_dir=np.array([0, 0]), color="darkgreen"),
            lambda ax, ax2: BackgroundObject(ax, ax2, v_l=3, v_w=4, x_offset=10, y_offset=30, vel_dir=np.array([0, 0]), color="slategray"),
        ],
        [
            {"suffix":"conf_1_it_0", "psi_0": np.pi/2, "phi_0": np.pi-np.pi/15, 
             "psi_phi_dot": lambda t, t_max: (0, 0)},
        ]
    ]
]

In [ ]:
###########################
### Wildlife Simulation ###
###########################

np.random.seed(5)
app_name = "Wildlife tracking"

config_id = 0
it_id = 0

conf_dict = sim_configs[config_id][2][it_id]

psi_0 = conf_dict["psi_0"]
phi_0 = conf_dict["phi_0"]

fig, ax, ax2, camera_fov, psi_line, psi_data, phi_line, \
phi_data, u1_line, u1_data, u2_line, u2_data = animation_setup(sensor_w, sensor_h, f, x_c, psi_0, phi_0, t_max, fps)

if isinstance(sim_configs[config_id][0][0], list):
    vehicles = [vehicle_f(ax,ax2) for vehicle_f in sim_configs[config_id][0][0]]
    vehicles2 = [vehicle_f(ax,ax2) for vehicle_f in sim_configs[config_id][0][1]]
else:
    vehicles = [vehicle_f(ax,ax2) for vehicle_f in sim_configs[config_id][0]]
    vehicles2 = []
background = [background_f(ax,ax2) for background_f in sim_configs[config_id][1]]
obj_detector = ObjectDetector(ax2, camera_fov, fps=fps, dpi=100)

psi_phi_dot = conf_dict["psi_phi_dot"]
phi_dots = np.array([psi_phi_dot(f/fps, t_max) for f in np.arange(fps*t_max)])
title = ax.set_title('PT Camera Simulation, time=0s\nApplication: {}'.format(app_name))

# Animate FOV
ani = animation.FuncAnimation(fig, update, t_max*fps+10, fargs=(conf_dict["suffix"],), interval=10, blit=False, repeat=False)
ani.save('simulations/PT_camera_simulation_wildlife_16_9_yolo_sort_classify_test.gif', fps=fps)#, writer='imagemagick')

#plt.savefig("simulations/PT_camera_simulation_wildlife_16_9_yolo_sort_classify_snapshot.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
#########################################
### Wildlife Simulation with Tracking ###
#########################################

app_name = "Wildlife tracking"

config_id = 1
it_id = 0

conf_dict = sim_configs[config_id][2][it_id]

psi_0 = conf_dict["psi_0"]
phi_0 = conf_dict["phi_0"]

fig, ax, ax2, camera_fov, psi_line, psi_data, phi_line, \
phi_data, u1_line, u1_data, u2_line, u2_data = animation_setup(sensor_w, sensor_h, f, x_c, psi_0, phi_0, t_max, fps)

if isinstance(sim_configs[config_id][0][0], list):
    vehicles = [vehicle_f(ax,ax2) for vehicle_f in sim_configs[config_id][0][0]]
    vehicles2 = [vehicle_f(ax,ax2) for vehicle_f in sim_configs[config_id][0][1]]
else:
    vehicles = [vehicle_f(ax,ax2) for vehicle_f in sim_configs[config_id][0]]
    vehicles2 = []
background = [background_f(ax,ax2) for background_f in sim_configs[config_id][1]]
obj_detector = ObjectDetector(ax2, camera_fov, fps=fps, dpi=100, threshold=0.2)

psi_phi_dot = conf_dict["psi_phi_dot"]
phi_dots = np.array([psi_phi_dot(f/fps, t_max) for f in np.arange(fps*t_max)])
title = ax.set_title('PT Camera Simulation, time=0s\nApplication: {}'.format(app_name))

# Animate FOV
ani = animation.FuncAnimation(fig, update_dynamic, t_max*fps+10, fargs=(conf_dict["suffix"],), interval=10, blit=False, repeat=False)
ani.save('simulations/PT_camera_simulation_wildlife_16_9_yolo_sort_classify_track_2x_test.gif', fps=fps/2)#, writer='imagemagick')

#plt.savefig("simulations/PT_camera_simulation_wildlife_16_9_yolo_sort_classify_track_snapshot3.png", dpi=300, bbox_inches="tight")
plt.show()